In [1]:
import pandas as pd
import numpy as np
import random
from ortools.sat.python import cp_model

In [2]:
def doub_row(row):
    hora_inicial = pd.to_datetime(row['Horário'], format='%H:%M:%S')
    
    nova_linha_1 = row.copy()
    nova_linha_2 = row.copy()
    
    nova_linha_1['Horário'] = (hora_inicial).strftime('%H:%M:%S')
    nova_linha_2['Horário'] = (hora_inicial + pd.Timedelta(hours=1)).strftime('%H:%M:%S')
    
    return pd.DataFrame([nova_linha_1, nova_linha_2])

def tri_row(row):
    hora_inicial = pd.to_datetime(row['Horário'], format='%H:%M:%S')
    
    nova_linha_1 = row.copy()
    nova_linha_2 = row.copy()
    nova_linha_3 = row.copy()
    
    nova_linha_1['Horário'] = (hora_inicial).strftime('%H:%M:%S')
    nova_linha_2['Horário'] = (hora_inicial + pd.Timedelta(hours=1)).strftime('%H:%M:%S')
    nova_linha_3['Horário'] = (hora_inicial + pd.Timedelta(hours=2)).strftime('%H:%M:%S')
    
    return pd.DataFrame([nova_linha_1, nova_linha_2, nova_linha_3])

def desaninhar_dias(df):
    df['Dias da Semana'] = df['Dias da Semana'].str.split(',')
    df = df.explode('Dias da Semana')
    return df

# Aplicar a função para todas as linhas triplas
def apply_func(df,func):
    linhas_triplicadas = df.apply(func, axis=1)
    return pd.concat(linhas_triplicadas.tolist(), ignore_index=True)

In [3]:
aulas_raw = pd.read_excel('/home/cayena/Downloads/ROTA TESTE CARLOS E ALLAN.xlsx',header=1)
aulas = aulas_raw.loc[~aulas_raw['Grupo'].isnull()][['Grupo','Horário','Dias da Semana']]

In [4]:
aulas = aulas.loc[aulas['Dias da Semana']!='FOLDER'].reset_index(drop=True)

In [5]:
aulas['Dias da Semana'] = aulas['Dias da Semana'].str.replace('EVERYDAY','2ª ● 3ª ● 4ª ● 5ª ● 6ª')

In [6]:
tri = aulas[aulas['Dias da Semana'] == 'Saturday - Triple'].copy()
doub = aulas[aulas['Dias da Semana'].str.contains('DOUBLE')].copy()
aulas_simples = aulas[~aulas['Dias da Semana'].str.contains('DOUBLE|Saturday - Triple')].copy()

In [7]:
# tratando os dados para colocar cada linha uma aula
aulas['Dias da Semana'] = aulas['Dias da Semana'].str.replace('EVERYDAY','2ª ● 3ª ● 4ª ● 5ª ● 6ª')

# separando as aulas que são triplas, duplas e o resto
tri = aulas.loc[aulas['Dias da Semana'] == 'Saturday - Triple']
doub = aulas.loc[aulas['Dias da Semana'].str.contains('DOUBLE')]
aulas_simples = aulas[~aulas['Dias da Semana'].str.contains('DOUBLE|Saturday - Triple')].copy()

# tratando a colunas horario
aulas_simples['Horário'] = pd.to_datetime(aulas_simples['Horário'],format='%H:%M:%S').dt.strftime('%H:%M:%S')

In [8]:
# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = apply_func(doub, doub_row)
aulas_triplicadas = apply_func(tri, tri_row)

# padronizando a coluna de dias da semana
aulas_duplicadas['Dias da Semana'] = aulas_duplicadas['Dias da Semana'].str.replace('DOUBLE','')
aulas_triplicadas['Dias da Semana'] = aulas_triplicadas['Dias da Semana'].str.replace(' - Triple','')

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

# desaninhando os dias da semana
df_tratado['Dias da Semana'] = df_tratado['Dias da Semana'].str.replace('●',',')
df_tratado['Dias da Semana'] = df_tratado['Dias da Semana'].str.split(',')
df_tratado = df_tratado.explode('Dias da Semana').reset_index(drop=True)

In [9]:
# Definindo nomes dos professores
Professores = {'Name':['Allan','Carlos','Henrique','Yasmin','Malu','Tanze',
                       'Okamoto','Edu','Wolf','Odin','Renan','Gabiraes',
                       'Nicolas','Novaes','Jorge','Kaori','Igor','Magnus',
                       'Nepo','Jurema','Zoe','Humberto','Luiz','Ricardo','Ian']}
teachers = pd.DataFrame(Professores)
teachers.shape


(25, 1)

In [10]:
all_teacher = teachers['Name'].to_list()
# Dicionário de alocações
alocacoes = {}

# Modelo
model = cp_model.CpModel()

for i in all_teacher:
    for _, row in df_tratado.iterrows():
        g = row['Grupo']
        h = row['Horário']
        d = row['Dias da Semana']
        alocacoes[(i,g,h,d)] = model.NewBoolVar(f"{i}_converinglesson_{g}_period_{h}_day_{d}")


In [11]:
for d in df_tratado['Dias da Semana'].unique():
    for h in df_tratado['Horário'].unique():
        for g in df_tratado['Grupo'].unique():
            chaves_validas = [(i,g,h,d) for i in all_teacher if (i,g,h,d) in alocacoes]
            if chaves_validas:
                model.AddExactlyOne(alocacoes[(i,g,h,d)] for i in all_teacher)

In [12]:
for d in df_tratado['Dias da Semana'].unique():
    for h in df_tratado['Horário'].unique():
        for i in all_teacher:
            chaves_validas = [(i, g, h, d) for g in df_tratado['Grupo'].unique() if (i, g, h, d) in alocacoes]
            if chaves_validas:
                # Garantir que o professor i só possa dar uma aula no mesmo horário e dia
                model.AddAtMostOne(alocacoes[(i, g, h, d)] for g in df_tratado['Grupo'].unique() if (i, g, h, d) in alocacoes)

In [13]:
for d in df_tratado['Dias da Semana'].unique():
    for g in df_tratado['Grupo'].unique():
        for i in all_teacher:
            chaves_validas = [(i, g, h, d) for g in df_tratado['Grupo'].unique() if (i, g, h, d) in alocacoes]
            if chaves_validas:
                # Garantir que o professor i só possa dar uma aula no mesmo horário e dia
                model.AddAtMostOne(alocacoes[(i, g, h, d)] for h in df_tratado['Horário'].unique() if (i, g, h, d) in alocacoes)

In [15]:
for i in all_teacher:
    for dia,horario in zip(df_tratado['Dias da Semana'],df_tratado['Horário']):
        turmas_no_mesmo_horario = [turma for turma, dia_turma, hora_turma in zip(df_tratado['Grupo'], df_tratado['Dias da Semana'], df_tratado['Horário'])
                                   if dia_turma == dia and hora_turma == horario]

In [23]:
df_tratado[df_tratado['Grupo'].isin(["GIBRALTAR PRESENCIAL","BRISBANE PRESENCIAL","POLAND PRESENCIAL","BARCELONA PRESENCIAL"])]

,Grupo,Horário,Dias da Semana
390,GIBRALTAR PRESENCIAL,08:00:00,Saturday
391,GIBRALTAR PRESENCIAL,09:00:00,Saturday
392,GIBRALTAR PRESENCIAL,10:00:00,Saturday
393,BRISBANE PRESENCIAL,08:00:00,Saturday
394,BRISBANE PRESENCIAL,09:00:00,Saturday
395,BRISBANE PRESENCIAL,10:00:00,Saturday
405,BARCELONA PRESENCIAL,08:00:00,Saturday
406,BARCELONA PRESENCIAL,09:00:00,Saturday
407,BARCELONA PRESENCIAL,10:00:00,Saturday
423,POLAND PRESENCIAL,08:00:00,Saturday


In [ ]:
turmas = ['Turma A','Turma A', 'Turma B','Turma B', 'Turma C','Turma C']
dias_da_semana = ['Segunda', 'Segunda', 'Segunda','terça','terça','terça'] 
horarios = ['08:00','09:00', '08:00', '10:00', '10:00', '11:00']            
professores = ['Prof. 1', 'Prof. 2', 'Prof. 3','Prof. 4']

In [16]:
turmas_no_mesmo_horario

['GIBRALTAR PRESENCIAL',
 'BRISBANE PRESENCIAL',
 'NORTHAMPTOM PRESENCIAL',
 'SOUTHAMPTON PRESENCIAL',
 'AUSTIN ONLINE',
 'BARCELONA PRESENCIAL',
 'CHARLESTON PRESENCIAL',
 'NEBRASKA PRESENCIAL',
 'SCOTT ONLINE',
 'MONTERREY ONLINE',
 'SAN JUAN ONLINE',
 'POLAND PRESENCIAL']

In [14]:
#Solução
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solução encontrada!")
    
    # Imprimir as alocações
    for i in all_teacher:
        for _, row in df_tratado.iterrows():
            g = row['Grupo']
            h = row['Horário']
            d = row['Dias da Semana']
            if solver.BooleanValue(alocacoes[(i,g,h,d)]):
                print(f"{i} está alocado para {g} no horário {h} no dia {d}")
else:
    print("Nenhuma solução viável foi encontrada.")

Solução encontrada!
Allan está alocado para GREENSBURG ONLINE no horário 07:00:00 no dia 2ª 
Allan está alocado para GREENSBURG ONLINE no horário 07:00:00 no dia  3ª 
Allan está alocado para GREENSBURG ONLINE no horário 07:00:00 no dia  5ª
Allan está alocado para CONVERSATION 14 PRESENCIAL no horário 08:00:00 no dia 3ª
Allan está alocado para KNOXVILLE PRESENCIAL no horário 08:00:00 no dia 2ª 
Allan está alocado para KNOXVILLE PRESENCIAL no horário 08:00:00 no dia  3ª 
Allan está alocado para KNOXVILLE PRESENCIAL no horário 08:00:00 no dia  4ª 
Allan está alocado para KNOXVILLE PRESENCIAL no horário 08:00:00 no dia  5ª
Allan está alocado para GREAT BRITAIN PRESENCIAL no horário 09:00:00 no dia  5ª
Allan está alocado para GARLAND PRESENCIAL no horário 18:00:00 no dia 2ª 
Allan está alocado para GARLAND PRESENCIAL no horário 18:00:00 no dia  3ª 
Allan está alocado para GARLAND PRESENCIAL no horário 18:00:00 no dia  5ª
Allan está alocado para TAMPA PRESENCIAL no horário 19:00:00 no dia 2ª

In [11]:
from ortools.sat.python import cp_model

# Criar o modelo
model = cp_model.CpModel()

# Dados fornecidos
turmas = ['Turma A','Turma A', 'Turma B','Turma B', 'Turma C','Turma C']
dias_da_semana = ['Segunda', 'Segunda', 'Segunda','terça','terça','terça'] 
horarios = ['08:00','09:00', '08:00', '10:00', '10:00', '11:00']            
professores = ['Prof. 1', 'Prof. 2', 'Prof. 3','Prof. 4']

# Criar um dicionário para armazenar as variáveis booleanas
alocacao = {}

# Criar as variáveis booleanas para cada combinação de professor, turma, dia e horário
for turma in turmas:
    for dia, horario in zip(dias_da_semana, horarios):
        for professor in professores:
            # Nome da variável
            nome_variavel = f'{professor}_{turma}_{dia}_{horario}'
            
            # Criar variável booleana (0 ou 1)
            alocacao[nome_variavel] = model.NewBoolVar(nome_variavel)

# Restrições
# 1. Um professor pode dar aula em no máximo uma turma em um dado dia e horário
for professor in professores:
    for dia, horario in zip(dias_da_semana, horarios):
        # Identificar todas as turmas com o mesmo dia e horário
        turmas_no_mesmo_horario = [turma for turma, dia_turma, hora_turma in zip(turmas, dias_da_semana, horarios)
                                   if dia_turma == dia and hora_turma == horario]
        
        # Garantir que o professor só possa ser alocado em no máximo uma turma nesse dia/horário
        model.Add(sum(alocacao[f'{professor}_{turma}_{dia}_{horario}'] for turma in turmas_no_mesmo_horario) <= 1)




In [13]:
turmas_no_mesmo_horario

['Turma C']

In [12]:
# Resolver o modelo
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Exibir as soluções
if status == cp_model.OPTIMAL:
    for turma, dia, horario in zip(turmas, dias_da_semana, horarios):
        for professor in professores:
            if solver.BooleanValue(alocacao[f'{professor}_{turma}_{dia}_{horario}']):
                print(f'{professor} foi alocado à {turma} no {dia} às {horario}.')
else:
    print("Não foi encontrada uma solução ótima.")

TESTE